In [1]:
import json
import regex as re

In [135]:
results = []
query_terms = ['per', 'se']
feedback = [1, 0, 1,0, 1, 0, 0, 0, 0, 0]

with open('per_se.json') as user_file:
    for line in user_file:
        result = json.loads(line)
        results.append(result)

In [136]:
results

[{'Title': 'Per Se | Thomas Keller Restaurant Group',
  'Link': 'https://www.thomaskeller.com/perseny',
  'Snippet': 'Per Se. Per Se Front Door. center. About. About; Restaurant · Team · Info & Directions · Gift Experiences · Reservations; Menus & Stories. Menus & Stories'},
 {'Title': 'Perse Definition & Meaning - Merriam-Webster',
  'Link': 'https://www.merriam-webster.com/dictionary/perse',
  'Snippet': 'The meaning of PERSE is of a dark grayish blue resembling indigo. How to use perse in a sentence. Did you know?'},
 {'Title': 'Per Se - New York, NY | Tock',
  'Link': 'https://www.exploretock.com/perse',
  'Snippet': "Opened in 2004, Per Se is Thomas Keller's acclaimed interpretation of The French Laundry in the Deutsche Bank Center at Columbus Circle."},
 {'Title': 'Per Se (@perseny) • Instagram photos and videos',
  'Link': 'https://www.instagram.com/perseny/',
  'Snippet': '268K Followers, 197 Following, 929 Posts - See Instagram photos and videos from Per Se (@perseny)'},
 {'Ti

In [137]:
pattern = re.compile(r'''
’s|’t|’re|’ve|’m|’ll|’d| ?\p{L}+| ?\p{N}+|\s+”
''')
# def get_words(string):

    

In [138]:
stop_words = set()
with open('../stop_words.txt', 'r') as stop_words_file:
    for line in stop_words_file:
        stop_words.add(line.strip())

In [142]:
import regex as re
pattern = re.compile(r'''
’s|’t|’re|’ve|’m|’ll|’d| ?\p{L}+| ?\p{N}+”
''')

def extract_words(result_list):
    documents = []
    vocab = set()
    for result in result_list:
        title = result.get('Title', '')
        snippet = result.get('Summary', '')
        title_words = re.findall(pattern, title.lower())
        title_words = [word.strip() for word in title_words if (word.strip() not in stop_words) or (word.strip() in query_terms)]
        snippet_words = re.findall(pattern, snippet.lower())
        snippet_words = [word.strip() for word in snippet_words if (word.strip() not in stop_words) or (word.strip() in query_terms)]
        vocab.update(title_words + snippet_words)
        documents.append({'title': title_words, 'summary': snippet_words})
    return documents, vocab

In [143]:
documents, vocab = extract_words(results)

In [144]:
documents

[{'title': ['per', 'se', 'thomas', 'keller', 'restaurant', 'group'],
  'summary': []},
 {'title': ['perse', 'definition', 'meaning', 'merriam', 'webster'],
  'summary': []},
 {'title': ['per', 'se', 'york', 'ny', 'tock'], 'summary': []},
 {'title': ['per', 'se', 'perseny', 'instagram', 'photos', 'videos'],
  'summary': []},
 {'title': ['per', 'se', 'york', 'michelin', 'guide', 'restaurant'],
  'summary': []},
 {'title': ['per', 'se', 'wiktionary', 'dictionary'], 'summary': []},
 {'title': ['per', 'se', 'definition', 'cambridge', 'english', 'dictionary'],
  'summary': []},
 {'title': ['james', 'perse', 'los', 'angeles'], 'summary': []},
 {'title': ['per', 'se', 'english', 'meaning', 'cambridge', 'dictionary'],
  'summary': []},
 {'title': ['per', 'se', 'wex', 'law', 'lii', 'legal', 'institute'],
  'summary': []}]

In [145]:
query_terms = ['per', 'se']

In [146]:
relevant_docs = set()

for doc in range(len(feedback)):
    if feedback[doc] == 1: 
        relevant_docs.add(doc)

In [147]:
relevant_docs

{0, 2, 4}

In [148]:
def is_query_term_in_window(words, index, query_terms, window_size):
    for i in range(index - window_size, index+window_size+1):
        if i < 0 or i>len(words) or i == index: 
            continue
        if words[i] in query_terms: 
            return True
    
    return False

In [149]:
inverse_list = {word:{} for word in vocab}
window_size = 2

for doc_i, document in enumerate(documents):
    doc_words = document['title'] + ['\\']*window_size+ document['summary']
    
    for word_i, word in enumerate(doc_words):
        if word == '\\': 
            continue

        word_dict = inverse_list[word]
        if doc_i not in inverse_list[word]:
            word_dict[doc_i] = {'frequency': 0, 'close_to_query': False}
        
        word_dict[doc_i]['frequency'] = word_dict[doc_i]['frequency']+1
        word_dict[doc_i]['close_to_query'] = is_query_term_in_window(doc_words, word_i, query_terms, window_size)




# word: {docs: {freq, close_to_query, }        

In [152]:
inverse_list['thomas']

{0: {'frequency': 1, 'close_to_query': True}}

In [153]:
all_documents = set(range(1, 10+1))

In [154]:
def gini(n_relevant_docs, n_irrelevant_docs):
        eps = 1e-7
        p_relevance = n_relevant_docs/(n_irrelevant_docs+n_relevant_docs+eps)
        p_irrelevance = n_irrelevant_docs/(n_irrelevant_docs+n_relevant_docs+eps)
        return 1 - p_irrelevance**2 - p_relevance**2

In [155]:
gini(18, 0)

1.1111111142270147e-08

In [156]:
# word = 'messi'

# docs_with_word = list(inverse_list[word])

# all_documents = set(range(0, len(feedback)))
# relevant_docs = set([i for i in range(len(feedback)) if feedback[i] == 1])

# docs_with_word = set(inverse_list[word].keys())
# docs_without_word = all_documents - docs_with_word

# n_relevant_with_word = len(docs_with_word.intersection(relevant_docs))
# n_irrelevant_with_word = len(docs_with_word)- n_relevant_with_word

# n_relevant_without_word = len(docs_without_word.intersection(relevant_docs))
# n_irrelevant_without_word = len(docs_without_word) - n_relevant_without_word

# w1 = len(docs_with_word)/len(all_documents)
# w2 = 1.0 - w1

# base_gini = gini(len(relevant_docs), len(all_documents - relevant_docs))
# word_gini = w1*gini(n_relevant_with_word, n_irrelevant_with_word) + w2*gini(n_relevant_without_word, n_irrelevant_without_word)

# gini_gain = base_gini-word_gini

KeyError: 'messi'

In [157]:
def gini_gain(word, inverse_list, feedback):
    docs_with_word = list(inverse_list[word])

    all_documents = set(range(0, len(feedback)))
    relevant_docs = set([i for i in range(len(feedback)) if feedback[i] == 1])

    docs_with_word = set(inverse_list[word].keys())
    docs_without_word = all_documents - docs_with_word

    n_relevant_with_word = len(docs_with_word.intersection(relevant_docs))
    n_irrelevant_with_word = len(docs_with_word)- n_relevant_with_word

    n_relevant_without_word = len(docs_without_word.intersection(relevant_docs))
    n_irrelevant_without_word = len(docs_without_word) - n_relevant_without_word

    w1 = len(docs_with_word)/len(all_documents)
    w2 = 1.0 - w1

    base_gini = gini(len(relevant_docs), len(all_documents - relevant_docs))
    word_gini = w1*gini(n_relevant_with_word, n_irrelevant_with_word) \
                + w2*gini(n_relevant_without_word, n_irrelevant_without_word)

    gini_gain = base_gini-word_gini
    return gini_gain

In [158]:
percentage_of_relevant_docs = {}

for word, docs in inverse_list.items():
    number_of_relevant_docs = 0
    for doc in docs:
        number_of_relevant_docs += feedback[doc - 1]
    percentage_of_relevant_docs[word] = number_of_relevant_docs/len(docs)

In [160]:
words_to_search

['group',
 'keller',
 'guide',
 'thomas',
 'ny',
 'michelin',
 'restaurant',
 'tock',
 'york']

In [161]:
ranking = {}

for word in words_to_search:
    ranking[word] = gini_gain(word, inverse_list, feedback)

In [162]:
ranking

{'group': 0.10888886740247217,
 'keller': 0.10888886740247217,
 'guide': 0.10888886740247217,
 'thomas': 0.10888886740247217,
 'ny': 0.10888886740247217,
 'michelin': 0.10888886740247217,
 'restaurant': 0.2449999759750016,
 'tock': 0.10888886740247217,
 'york': 0.2449999759750016}

In [163]:
from math import log

In [164]:
for word in words_to_search:
    word_docs = set(inverse_list[word].keys())
    relevant_word_docs = word_docs.intersection(relevant_docs)
    total_relevant_freq = 0
    for doc in relevant_word_docs:
        total_relevant_freq += inverse_list[word][doc]['frequency']
    

    ranking[word]+= log(1.0 + total_relevant_freq)


In [165]:
for word in words_to_search:
    word_docs = set(inverse_list[word].keys())
    relevant_word_docs = word_docs.intersection(relevant_docs)
    total_close_to_query_occurences = 0
    for doc in relevant_word_docs:
        if inverse_list[word][doc]['close_to_query']:
            total_close_to_query_occurences+=1
    
    ranking[word] += log(1.0+total_close_to_query_occurences)

In [166]:
ranking

{'group': 0.8020360479624175,
 'keller': 1.4951832285223627,
 'guide': 0.8020360479624175,
 'thomas': 1.4951832285223627,
 'ny': 1.4951832285223627,
 'michelin': 1.4951832285223627,
 'restaurant': 1.3436122646431115,
 'tock': 0.8020360479624175,
 'york': 2.4422245533112212}

In [18]:
res = sorted(words_to_search, key= lambda x: ranking[x])

In [19]:
{word: ranking[word] for word in res}

{'wikipedia': -0.05,
 'messi': -0.05,
 'french': 0.0,
 'paris': 0.0,
 'germain': 0.0,
 'signed': 0.0,
 'club': 0.0,
 'captaincy': 0.0,
 'assumed': 0.0,
 'record': 0.0,
 'sixth': 0.0,
 'contract': 0.0,
 'saint': 0.0,
 'barcelona': 0.0,
 'ballon': 0.0}

In [20]:
res

['wikipedia',
 'messi',
 'french',
 'paris',
 'germain',
 'signed',
 'club',
 'captaincy',
 'assumed',
 'record',
 'sixth',
 'contract',
 'saint',
 'barcelona',
 'ballon']